Motivation: Need a way to choose between machine learning models
Goal is to estimate likely performance of a model on out-of-sample data
Initial idea: Train and test on the same data
But, maximizing training accuracy rewards overly complex models which overfit the training data
Alternative idea: Train/test split
Split the dataset into two pieces, so that the model can be trained and tested on different data
Testing accuracy is a better estimate than training accuracy of out-of-sample performance
But, it provides a high variance estimate since changing which observations happen to be in the testing set can significantly change testing accuracy

In [2]:
# all of the imports
import pandas as pd
import numpy as np
import pickle 
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
% matplotlib inline

import warnings
warnings.filterwarnings('ignore')

/Users/codess/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
#Load data

udemy_df = pd.read_pickle("udemy_regression_df.pkl") #986 records
udemy_df.head(4)

,X1,X2,X3,X4,X5,X6,Y,X7,X8,X9,X10,X11,X12
0,61,11.5,10.99,19.99,4.7,12,76,6,0,0,1,0,0
1,24,2.5,10.99,49.99,4.4,18,84,5,0,0,0,0,1
2,44,2.5,10.99,199.99,4.5,24,535,6,0,0,1,0,0
3,11,1.0,10.99,49.99,4.3,37,119,6,0,1,0,0,0


In [4]:
y = udemy_df['Y']
y.head()

0      76
1      84
2     535
3     119
4    2729
Name: Y, dtype: int64

In [5]:
X = udemy_df.drop(columns=['Y'])

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 986 entries, 0 to 1286
Data columns (total 12 columns):
X1     986 non-null int64
X2     986 non-null float64
X3     986 non-null float64
X4     986 non-null float64
X5     986 non-null float64
X6     986 non-null int64
X7     986 non-null int64
X8     986 non-null uint8
X9     986 non-null uint8
X10    986 non-null uint8
X11    986 non-null uint8
X12    986 non-null uint8
dtypes: float64(4), int64(3), uint8(5)
memory usage: 66.4 KB


Cross-Validated Regression
As you probably guessed, the problem with our model above was that we trained and tested our model on the same dataset. This means our model could be very likely to overfit and not perform as well when it tries to generalize to real world data, and after all generalization is the key to machine learning.
Thus, we have a need for cross-validation in our model evaluation process. That is, we need to find a way to train our model on 1 randomly chosen set of data and evaluate it against a separate random test set. Thankfully sklearn provides a bevy of built-in ways to perform cross-validation.
Cross-Validation with sklearn
The simplest way to perform cross-validation with an sklearn model is to have it perform a random train/test split of the data for you. It's customary to use something like 2/3 of your data for training, and the remaining 1/3 for testing.
sklearn's train_test_split function provides exactly that. Here's an example of it in action.

In [7]:
lr = LinearRegression()
# INSTRUCTOR NOTE: Run this multiple times to show the variation
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# Fit the model against the training data
lr.fit(X_train, y_train)
# Evaluate the model against the testing data
lr.score(X_test, y_test)

0.8809552229416948

Comparing cross-validation to train/test split
Advantages of cross-validation:
More accurate estimate of out-of-sample accuracy
More "efficient" use of data (every observation is used for both training and testing)
Advantages of train/test split:
Runs K times faster than K-fold cross-validation
Simpler to examine the detailed results of the testing process
Cross-validation recommendations
K can be any number, but K=10 is generally recommended
For classification problems, stratified sampling is recommended for creating the folds
Each response class should be represented with equal proportions in each of the K folds
scikit-learn's cross_val_score function does this by default

In [9]:

model = sm.OLS(y_train,X_train) # for statsmodels 
# Fit your model to your training set
fit = model.fit()
# Print summary statistics of the model's performance
fit.summary()

#######
#y, X = patsy.dmatrices('Y ~ X1 + X2 + X3 + X4 + X5 + X6', data=df, return_type="dataframe")
#statsmodels -> better than scikit learn for linear reg
# df -> degree of freedom : no of predictors

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     1031.
Date:                Sun, 22 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:15:54   Log-Likelihood:                -6765.4
No. Observations:                 690   AIC:                         1.355e+04
Df Residuals:                     678   BIC:                         1.361e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
X1             0.1065      4.112      0.026      0.979      -7.966       8.180
X2            33.1968     23.324      1.423      0.155     -12.599      78.993
X3            -9.6077      8.212     -1.170      0.242     -25.732       6.516
X4             5.6409      2.736      2.062      0.040       0.268      11.014
X5           377.7637    175.874      2.148      0.032      32.440     723.087
X6             5.6162      0.057     97.786      0.000       5.503       5.729
X7          -280.6232    221.406     -1.267      0.205    -715.347     154.101
X8          -751.2295   4546.627     -0.165      0.869   -9678.390    8175.931
X9           880.1227   1289.563      0.682      0.495   -1651.894    3412.139
X10          612.1546   1277.014      0.479      0.632   -1895.222    3119.531
X11          523.7717   1596.968      0.328      0.743   -2611.826    3659.369
X12          938.4213   1291.253      0.727      0.468   -1596.914    3473.757
==============================================================================
Omnibus:                      747.216   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           103011.381
Skew:                           4.736   Prob(JB):                         0.00
Kurtosis:                      62.104   Cond. No.                     8.76e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.76e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

RENAME:
Y = Enrollment
X1 = Lectures
X2 = Hours
X3 = Sale
X4 = Price
X5 = Stars
X6 = Number of Ratings
X7 = Incentives
X8 = No Level
X9 = All Levels
X10 = Beginner
X11 = Expert
X12 = Intermediate

In [10]:
X_train.head(5)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12
600,10,1.0,10.99,19.99,2.6,5,4,0,0,1,0,0
48,39,1.5,10.99,119.99,4.6,9,6,0,1,0,0,0
701,18,4.0,10.99,39.99,4.1,249,6,0,1,0,0,0
855,90,8.0,10.99,19.99,3.5,381,5,0,1,0,0,0
537,71,2.5,10.99,49.99,3.7,7,6,0,1,0,0,0


AttributeError: 'Series' object has no attribute 'train'

In [16]:
df = X_train

In [15]:
df.head(4)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,1268,1270,1271,1272,1273,1275,1281,1282,1284,1286
600,10.0,1.0,10.99,19.99,2.6,5.0,4.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,39.0,1.5,10.99,119.99,4.6,9.0,6.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,18.0,4.0,10.99,39.99,4.1,249.0,6.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
855,90.0,8.0,10.99,19.99,3.5,381.0,5.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df['Y'] = y_train

In [22]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,y,Y
600,10,1.0,10.99,19.99,2.6,5,4,0,0,1,0,0,523,523
48,39,1.5,10.99,119.99,4.6,9,6,0,1,0,0,0,72,72
701,18,4.0,10.99,39.99,4.1,249,6,0,1,0,0,0,1994,1994
855,90,8.0,10.99,19.99,3.5,381,5,0,1,0,0,0,4073,4073
537,71,2.5,10.99,49.99,3.7,7,6,0,1,0,0,0,99,99


In [23]:
# Create your feature matrix (X) and target vector (y)
y_train, X_train = patsy.dmatrices('Y ~ X1 + X2 + X3 + X4 + X5 + X6 + X7', data=df, return_type="dataframe")
# implicitly add the beta constatnt term 
# patsy is the interpreter to read the string | and creates the column of 1s for the constant term in the matrix]
# Create your model
model = sm.OLS(y_train,X_train) # for statsmodels 
# Fit your model to your training set
fit = model.fit()
# Print summary statistics of the model's performance
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     1627.
Date:                Sun, 22 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:40:30   Log-Likelihood:                -6765.8
No. Observations:                 690   AIC:                         1.355e+04
Df Residuals:                     682   BIC:                         1.358e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    756.2769   1249.125      0.605      0.545   -1696.315    3208.869
X1             0.1411      4.093      0.034      0.973      -7.895       8.177
X2            32.8692     23.224      1.415      0.157     -12.731      78.469
X3            -9.1392      8.156     -1.121      0.263     -25.152       6.874
X4             5.6204      2.723      2.064      0.039       0.274      10.967
X5           363.9760    172.527      2.110      0.035      25.229     702.723
X6             5.6186      0.057     98.249      0.000       5.506       5.731
X7          -260.8685    219.398     -1.189      0.235    -691.645     169.908
==============================================================================
Omnibus:                      749.169   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           103677.424
Skew:                           4.757   Prob(JB):                         0.00
Kurtosis:                      62.293   Cond. No.                     2.41e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.41e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
df = X_train

In [26]:
df['Y'] = y_train

In [27]:
# Create your feature matrix (X) and target vector (y)
y_train, X_train = patsy.dmatrices('Y ~ + X2 + X4 + X5 + X6', data=df, return_type="dataframe")
# implicitly add the beta constatnt term 
# patsy is the interpreter to read the string | and creates the column of 1s for the constant term in the matrix]
# Create your model
model = sm.OLS(y_train,X_train) # for statsmodels 
# Fit your model to your training set
fit = model.fit()
# Print summary statistics of the model's performance
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.943
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     2848.
Date:                Sun, 22 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:43:46   Log-Likelihood:                -6767.1
No. Observations:                 690   AIC:                         1.354e+04
Df Residuals:                     685   BIC:                         1.357e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -607.1174    772.290     -0.786      0.432   -2123.457     909.222
X2            30.9501     20.680      1.497      0.135      -9.653      71.553
X4             4.6203      2.506      1.844      0.066      -0.300       9.541
X5           362.4594    166.843      2.172      0.030      34.874     690.045
X6             5.6214      0.056    100.621      0.000       5.512       5.731
==============================================================================
Omnibus:                      754.655   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           105914.311
Skew:                           4.816   Prob(JB):                         0.00
Kurtosis:                      62.927   Cond. No.                     1.50e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.5e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
Fitting